In [6]:
import tensorflow as tf
import numpy as np
import os
import time

## 讀取數據

In [7]:
path_to_file = "./I am a CAT.txt"

In [8]:
# py2compat読み取り
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# テキストの長さは、テキストの文字数を指します
print ('Length of text: {} characters'.format(len(text)))

Length of text: 323574 characters


In [9]:
# テキストの最初の250文字を見てください
print(text[:250])

吾輩は猫である
夏目漱石
一
　吾輩は猫である。名前はまだ無い。
　どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかり


In [10]:
# テキスト内の繰り返しのない文字
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

3058 unique characters


# テキストの処理
## ベクトル化されたテキスト
トレーニングの前に、文字列を数値表現値にマップする必要があります。 2つのルックアップテーブルを作成します。1つは文字を数字にマップし、もう1つは数字を文字にマップします。

In [11]:
# 繰り返されない文字からインデックスへのマッピングを作成する
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [12]:
# 各文字には整数値があります。 文字をインデックス0からlen（unique）にマップすることに注意してください。
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  "'" :   3,
  '(' :   4,
  ')' :   5,
  '-' :   6,
  '.' :   7,
  '0' :   8,
  '1' :   9,
  '2' :  10,
  '3' :  11,
  '4' :  12,
  '6' :  13,
  '7' :  14,
  '8' :  15,
  '=' :  16,
  'A' :  17,
  'D' :  18,
  'E' :  19,
  ...
}


In [13]:
# テキストの最初の13文字の整数マッピングを表示します

print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'吾輩は猫である\r\n夏目漱石' ---- characters mapped to int ---- > [ 587 2646  103 1770   95   63  131    1    0  739 1888 1662 1925]


## 予測タスク
文字または文字のシーケンスが与えられた場合、次に可能性の高い文字は何ですか？ これは、モデルをトレーニングするために実行する必要のあるタスクです。 モデルへの入力は文字のシーケンスであり、出力を予測するようにモデルをトレーニングします。各タイムステップで、次の文字がどうなるかを予測します。

RNNは以前に見た要素に基づいて内部状態を維持するため、この時点で計算されたすべての文字を考えると、次の文字は何ですか？

## トレーニングサンプルとターゲットを作成する
次に、テキストをサンプルシーケンスに分割します。 各入力シーケンスには、テキストにseq_length文字が含まれています。

各入力シーケンスについて、対応するターゲットには同じ長さのテキストが含まれていますが、1文字右にシフトされています。

テキストを長さseq_length + 1のテキストブロックに分割します。 たとえば、seq_lengthが4で、テキストが「Hello」の場合、入力シーケンスは「Hell」になり、ターゲットシーケンスは「ello」になります。
これを行うには、最初にtf.data.Dataset.from_tensor_slices関数を使用して、テキストベクトルを文字インデックスストリームに変換します。

In [14]:
# 各入力文の最大長を設定します
seq_length = 100
examples_per_epoch = len(text)//seq_length

# トレーニングサンプル/ターゲットを作成する
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

吾
輩
は
猫
で


In [15]:
# batch では、1文字を必要な長さのシーケンスに簡単に変換できます。
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'吾輩は猫である\r\n夏目漱石\r\n一\r\n\u3000吾輩は猫である。名前はまだ無い。\r\n\u3000どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人'
'間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいと'
'も思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思'
'った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そう'
'してその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。\r\n\u3000この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくす'


In [16]:
# シーケンスごとに、mapメソッドを使用してコピーし、次に進んで入力テキストとターゲットテキストを作成します。 mapメソッドは、各バッチに単純な関数を適用できます。
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [18]:
# サンプルの最初のバッチの入力値とターゲット値を印刷します：
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '吾輩は猫である\r\n夏目漱石\r\n一\r\n\u3000吾輩は猫である。名前はまだ無い。\r\n\u3000どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて'
Target data: '輩は猫である\r\n夏目漱石\r\n一\r\n\u3000吾輩は猫である。名前はまだ無い。\r\n\u3000どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人'


これらのベクトルの各インデックスは、タイムステップとして処理されます。 タイムステップ0への入力として、モデルは「F」のインデックスを受け取り、次の文字として「i」のインデックスを予測しようとします。 次のタイムステップでは、モデルは同じ操作を実行しますが、RNNは現在の入力文字だけでなく、前のステップからの情報も考慮します。

In [19]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 587 ('吾')
  expected output: 2646 ('輩')
Step    1
  input: 2646 ('輩')
  expected output: 103 ('は')
Step    2
  input: 103 ('は')
  expected output: 1770 ('猫')
Step    3
  input: 1770 ('猫')
  expected output: 95 ('で')
Step    4
  input: 95 ('で')
  expected output: 63 ('あ')


## トレーニングバッチを作成する
以前は、tf.dataを使用して、テキストを管理可能なシーケンスに分割しました。 ただし、このデータをモデルに送信する前に、データをシャッフルしてバッチにパックする必要があります。

In [20]:
# バッチサイズ
BATCH_SIZE = 64

# データセットを再配置するためのバッファサイズを設定します
#（TFデータは、潜在的に無制限のシーケンスを処理するように設計されています。
# したがって、メモリ内のシーケンス全体を再配置しようとはしません。 それどころか、
# バッファを維持し、バッファ内の要素を再配置します。 ）。
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## モデルを作成する
tf.keras.Sequentialを使用してモデルを定義します。 この簡単な例では、3つのレイヤーを使用してモデルを定義しました。

* tf.keras.layers.Embedding：入力レイヤー。 各文字の数をembedding_dim次元のベクトルにマップするトレーニング可能な比較テーブル。
* tf.keras.layers.GRU：サイズがunits = rnn_unitsで指定されているRNNのタイプ（ここでLSTMレイヤーを使用することもできます）。
* tf.keras.layers.Dense：vocab_size出力を備えた出力レイヤー。

In [21]:
# 単語セットの長さ
vocab_size = len(vocab)

# 埋め込まれた寸法
embedding_dim = 256

# RNNユニットの数
rnn_units = 1024

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [23]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 3058) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           782848    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 3058)          3134450   
Total params: 7,855,602
Trainable params: 7,855,602
Non-trainable params: 0
_________________________________________________________________


In [26]:

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [27]:
sampled_indices

array([1347, 2761, 2216, 2195, 3027, 1180, 1517, 1371,  992, 1746, 2447,
       2715, 1298, 1306, 1385,  657, 2084, 1048,  120, 2335,  399, 2899,
       1497, 1645, 2114,  149, 1005, 1838, 2833, 1794, 2404,  846,  313,
       2514,  451, 1698,  197, 1284, 2885,  911, 1891,  341,  963, 1542,
        987, 1346, 1387,   11, 2523, 1938,  162, 1423, 2990, 1056, 1306,
        478, 2345,  321, 2215,  226, 2507, 2772, 1282, 2227, 2304,  413,
        598, 1539,  769,   31,   33, 2201, 1801,  854, 1487, 2010,  902,
       1014, 1139,  500,  579, 2252, 3020, 1485, 1753, 1866,  447, 2810,
       3001, 1647,  766, 1870,  568, 2310, 2818, 1108, 1766, 1633, 2514,
        691], dtype=int64)

In [28]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '、さすがに独仙君は鎌倉へ行って万年漬を食っただけあって、物に動じないね。どうも敬々服々だ。碁はまずいが、度胸は据ってる」\r\n「だから君のような度胸のない男は、少し真似をするがいい」と主人が後ろ向のまま'

Next Char Predictions: 
 '普鈴胡肉鼓拓武更弁牡西避敷斤期噂素念む蔭兜頻機滅綺ギ張疎陽琳衒宿併誹利炭ム放韻崇相倒幽水延晩朧3諷碗セ架鳴怨斤功薔侮胞与誡銷改脱菜冉呼氏如gi肢瑩寞槽童岳彷成匆君臓麻槍犯癬列闖鵯滑女白号萎陌慕狽溌誹坑'


In [29]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 3058)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       8.025424


In [30]:
model.compile(optimizer='adam', loss=loss)

In [31]:
# 檢查點保存至的目錄
checkpoint_dir = './training_checkpoints_2'

# 檢查點的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [32]:
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
EPOCHS=10

In [34]:

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
50/50 [==============================] - 96s 2s/step - loss: 5.7043
Epoch 2/10
50/50 [==============================] - 98s 2s/step - loss: 4.9302
Epoch 3/10
50/50 [==============================] - 99s 2s/step - loss: 4.3893
Epoch 4/10
50/50 [==============================] - 102s 2s/step - loss: 4.1184
Epoch 5/10
50/50 [==============================] - 99s 2s/step - loss: 3.9479
Epoch 6/10
50/50 [==============================] - 101s 2s/step - loss: 3.8195
Epoch 7/10
50/50 [==============================] - 102s 2s/step - loss: 3.7156
Epoch 8/10
50/50 [==============================] - 104s 2s/step - loss: 3.6300
Epoch 9/10
50/50 [==============================] - 106s 2s/step - loss: 3.5521
Epoch 10/10
50/50 [==============================] - 99s 2s/step - loss: 3.4826


# テキストを生成する
## 最新のチェックポイントを復元する
この予測ステップを単純に保つために、バッチサイズを1に設定します。 RNN状態がタイムステップからタイムステップに転送される方法により、モデルの構築後は固定バッチサイズのみが受け入れられます。 異なるbatch_sizeでモデルを実行するには、モデルを再構築し、チェックポイントから重みを復元する必要があります。

In [45]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_2\\ckpt_10'

In [46]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [47]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            782848    
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 3058)           3134450   
Total params: 7,855,602
Trainable params: 7,855,602
Non-trainable params: 0
_________________________________________________________________


# 予測サイクル
次のコードブロックはテキストを生成します。

* 最初に開始文字列を設定し、RNN状態を初期化し、生成する文字数を設定します。

* 開始文字列とRNN状態を使用して、次の文字の予測分布を取得します。

* 次に、分類分布を使用して、予測された文字のインデックスが計算されます。 この予測された文字をモデルへの次の入力として使用します。

* モデルによって返されたRNN状態は、モデルに送り返されます。 これで、モデルには、1つの文字だけでなく、学習するコンテキストが増えました。 次の文字を予測した後、変更されたRNN状態がモデルに再度返送されます。 モデルはこのようなもので、以前に予測されたキャラクターから常により多くのコンテキストを取得することによって学習します。

In [48]:
def generate_text(model, start_string):
    # 評価手順（学習したモデルを使用してテキストを生成します）

    # 生成される文字数
    num_generate = 1000

    # 開始文字列を数値に変換する（ベクトル化）
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # 結果を格納するために空の文字列が使用されます
    text_generated = []

    # 低温はより予測可能なテキストを生成します
    # 温度が高くなると、より驚くべきテキストが生成されます
    # 実験して最適な設定を見つけることができます
    temperature = 1.0

    # バッチサイズは1です。
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # バッチの寸法を削除します
        predictions = tf.squeeze(predictions, 0)

        # カテゴリ分布予測モデルによって返される文字
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 予測された文字と前の非表示状態を次の入力としてモデルに渡します
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [49]:
print(generate_text(model, start_string=u"吾輩は猫"))

吾輩は猫と令嬢であろう、腹で道望と猫がへえて、その辺の実をそれは道用がったら徒とこの何でも是非歌h T要も云い本したっと云う気がない。実は奥晶の消えて、長を楯にして云う。吾輩はどうも読んで、ためなくて、これは利かなる結論だ。しかし、平均の眼を通りつくなくなるの風が」
「どこしいかくきで別切と云けぬ。さった。
「この時はこの暁であるかれかならんと心配畠の布団の二三襦色が往来ても心を開き得て催いいの。人も純なものか」
「主人甜郷灯も這入らずに妨るない」
「何に尾主歌を製化して糞がなぜんじゃんでする。主人は鉄山の上でもうと思うようがは聳えなかわないっしっとはこれじゃないとらやって女子と云っている越え牧坪ば白いだろ、さんで今なにこの鏘すで行ったが、相違あってから拝見順のは論ちに毛を敲くようならべく見受呑心などなは妻にやろまいうなどほか分知れてい」
「これは二ついくと歯のである。このあと今なはあるの観かなくだって近物の演桶の御曰く落ちのように上らんが、その知らずるようにむては月並と評しは人衣を入らずかろせない。球である赤地でに一取別のと行く分らない、すでもたり、四つの講ジプ上であるなものみば池を食ったとかろか。主人は世法を突然人間になる。吾輩が耳の新深湯寺の嫌いじゃないかめものだろう。しか椽側かある。
「あれは不平君はたのかうか、そうだがっでしかければしょっこのと足もあと主人の犬に同じを地より度もまく存価している。
「うんよ猫の二円と主人の気色の法す水草を放逐なるところが「いようそうだから分ると前から喜滑を攻撃よるともち英快そうだろうが、三分日火者などこへ感じたって愛の説劇だそうだ。寒月君で聞見たっためと象の逆からいのなものか、二客だけだたのはなのじゃあれると思う。「あいく事もない。
「冗談しか、三間の人にかってハーゼ縊りなるのは世際な世のだっては舐めんよ。いってもって同毛をして癒らると泣きるから、ごとく楽不分から人勢だが流は捕え抜けりばかりが頭を借る布体なるほどいであの猫など同じた遠らは冗三四動器を隔たる。彼れらく年にやりとするも、少しのさ、僕ばかって…―無儀を、いつかろ」
「しい字野さん、するのところがあるが彼等朝君ら」と主人学性を貰いたまあ逆上術ジジクの人は睨めも入れない動をしても忘れる、湯に両人、運動には一つもないか、一人が、胸の以外有して見る。気薫もない。
